In [1]:
!git clone https://github.com/Alpha-VLLM/LLaMA2-Accessory.git

Cloning into 'LLaMA2-Accessory'...
remote: Enumerating objects: 4047, done.
remote: Counting objects: 100% (1350/1350), done.
remote: Compressing objects: 100% (476/476), done.
remote: Total 4047 (delta 1013), reused 966 (delta 870), pack-reused 2697 (from 1)
Receiving objects: 100% (4047/4047), 87.63 MiB | 22.67 MiB/s, done.
Resolving deltas: 100% (2563/2563), done.


In [2]:
%cd LLaMA2-Accessory


/kaggle/working/LLaMA2-Accessory


In [3]:
# install as package
!pip install -e .

Obtaining file:///kaggle/working/LLaMA2-Accessory
  Preparing metadata (setup.py) ... done
  Running setup.py develop for LLaMA2-Accessory


In [89]:
!pip install flash-attn --no-build-isolation
!pip install fairscale
!pip install huggingface_hub
!pip install sentencepiece
!pip install open_clip_torch

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.0/6.0 MB 17.7 MB/s eta 0:00:0000:0100:01
  Preparing metadata (setup.py) ... done
  Created wheel for flash-attn: filename=flash_attn-2.7.4.post1-cp311-cp311-linux_x86_64.whl size=187815463 sha256=d944fc7d2f962bce83fc4708c2fc0c21eaf8255962a0b350ae919362a51b7ef2
  Stored in directory: /root/.cache/pip/wheels/3d/88/d8/284b89f56af7d5bf366b10d6b8e251ac8a7c7bf3f04203fb4f
Successfully built flash-attn


In [4]:
# Import the `notebook_login` function from the `huggingface_hub` module.
from huggingface_hub import notebook_login

# Execute the `notebook_login` function to log in to the Hugging Face Hub.
notebook_login()

In [9]:
# Import the necessary function from the huggingface_hub library
from huggingface_hub import hf_hub_download

# Define the repository information
# repo_id = "meta-llama/Meta-Llama-3-8B"
repo_id = "meta-llama/Llama-2-7b"
# subfolder = "original"  # Specify the subfolder within the repository

# List of filenames to download
filenames = ["params.json", "tokenizer.model", "consolidated.00.pth"] 

# Specify the directory where you want to save the downloaded files
save_directory = "llama-2-7b/"  # Replace with your desired path

# Download each file
for filename in filenames:
    hf_hub_download(
        repo_id=repo_id,       # Repository ID
        filename=filename,     # Name of the file to download
        # subfolder=subfolder,   # Subfolder within the repository
        local_dir=save_directory  # Directory to save the downloaded file
    )

params.json:   0%|          | 0.00/102 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

consolidated.00.pth:   0%|          | 0.00/13.5G [00:00<?, ?B/s]

In [ ]:
# https://huggingface.co/meta-llama/Llama-2-7b/resolve/main/params.json

In [10]:
ls

accessory/           Large-DiT-T2I/              llama-3-8B/
asset/               LICENSE_llama2              README.md
data_example/        light-eval/                 requirements.txt
docs/                llama-2-7b/                 setup.py
Large-DiT-ImageNet/  LLaMA2_Accessory.egg-info/  SPHINX/


In [11]:
cat /kaggle/working/LLaMA2-Accessory/accessory/configs/data/finetune/sg/alpaca.yaml

META:
  -
    path: '../data/alpaca_gpt4_data.json'
    type: 'text'

In [14]:
import yaml
import os

# Load YAML file
file_path = "/kaggle/working/LLaMA2-Accessory/accessory/configs/data/finetune/sg/alpaca.yaml"
with open(file_path, 'r') as f:
    data = yaml.safe_load(f)

# Update path
data['META'][0]['path'] = '/kaggle/input/llama2-instruction-finetuning-data/alpaca_gpt4_data.json'

# Save back
with open(file_path, 'w') as f:
    yaml.dump(data, f, default_flow_style=False)

In [15]:
cat /kaggle/working/LLaMA2-Accessory/accessory/configs/data/finetune/sg/alpaca.yaml

META:
- path: /kaggle/input/llama2-instruction-finetuning-data/alpaca_gpt4_data.json
  type: text


In [29]:
!/kaggle/working/LLaMA2-Accessory/accessory/exps/finetune/sg/alpaca_llamaPeft_normBiasLora.sh \
  /kaggle/working/LLaMA2-Accessory/llama-2-7b/consolidated.00.pth \
  /kaggle/working/LLaMA2-Accessory/llama-2-7b/params.json \
  /kaggle/working/LLaMA2-Accessory/llama-2-7b/tokenizer.model

/bin/bash: line 1: /kaggle/working/LLaMA2-Accessory/accessory/exps/finetune/sg/alpaca_llamaPeft_normBiasLora.sh: Permission denied


In [21]:
pwd

'/kaggle/working/LLaMA2-Accessory'

In [32]:
!chmod +x /kaggle/working/LLaMA2-Accessory/accessory/exps/finetune/sg/alpaca_llamaPeft_normBiasLora.sh

In [36]:
!chmod +x /kaggle/working/LLaMA2-Accessory/accessory/exps/finetune/sg/alpaca_llamaPeft_normBiasLora.sh

In [34]:
cat /kaggle/working/LLaMA2-Accessory/accessory/exps/finetune/sg/alpaca_llamaPeft_normBiasLora.sh

#!/bin/bash

pretrained_path=$1
pretrained_type=meta_ori
llama_config="$2 configs/model/finetune/sg/llamaPeft_normBiasLora.json"
tokenizer_path="$3"
data_config=configs/data/finetune/sg/alpaca.yaml

data_parallel=sdp
model_parallel=1

exp_name=finetune/sg/alpaca_llamaPeft_normBiasLora
echo "exp name: $exp_name"
mkdir -p output/"$exp_name"

torchrun --master_port=1112 --nproc_per_node=8 main_finetune.py \
--output_dir output/"$exp_name" --epochs 4 --warmup_epochs 1 \
--batch_size 4 --accum_iter 2 --num_workers 4 \
--max_words 512 \
--lr 0.00005 --min_lr 0.000005 --clip_grad 2 --weight_decay 0.02 \
--data_parallel "$data_parallel" --model_parallel_size "$model_parallel" --checkpointing \
--llama_type llama_peft --llama_config $llama_config --tokenizer_path "$tokenizer_path" \
--no_visual \
--pretrained_path "$pretrained_path" --pretrained_type="$pretrained_type" \
--data_config $data_config \
2>&1 | tee -a output/"$exp_name"/output.log

echo "exp name: $exp_name"

In [35]:
cat /kaggle/working/LLaMA2-Accessory/accessory/configs/model/finetune/sg/llamaPeft_biasLora.json

{"lora_rank": 16, "bias_tuning": true}

In [84]:
import os

# Define the new script content
new_script_content = """#!/bin/bash

pretrained_path=$1
pretrained_type=meta_ori
llama_config="$2 configs/model/finetune/sg/llamaPeft_normBiasLora.json"
tokenizer_path="$3"
data_config=configs/data/finetune/sg/alpaca.yaml

data_parallel=sdp
model_parallel=1

exp_name=finetune/sg/alpaca_llamaPeft_normBiasLora
echo "exp name: $exp_name"
mkdir -p output/"$exp_name"

torchrun --master_port=1112 --nproc_per_node=1 /kaggle/working/LLaMA2-Accessory/accessory/main_finetune.py \\
--output_dir output/"$exp_name" --epochs 4 --warmup_epochs 1 \\
--batch_size 4 --accum_iter 2 --num_workers 4 \\
--max_words 512 \\
--lr 0.00005 --min_lr 0.000005 --clip_grad 2 --weight_decay 0.02 \\
--data_parallel "$data_parallel" --model_parallel_size "$model_parallel" --checkpointing \\
--llama_type llama_peft --llama_config $llama_config --tokenizer_path "$tokenizer_path" \\
--no_visual \\
--pretrained_path "$pretrained_path" --pretrained_type="$pretrained_type" \\
--data_config $data_config \\
2>&1 | tee -a output/"$exp_name"/output.log

echo "exp name: $exp_name"
"""

# Path to your shell script
script_path = "/kaggle/working/LLaMA2-Accessory/accessory/exps/finetune/sg/alpaca_llamaPeft_normBiasLora.sh"

# Write the new content to the file
with open(script_path, 'w') as f:
    f.write(new_script_content)

# Make the script executable
os.chmod(script_path, 0o755)  # Equivalent to chmod +x

print(f"Successfully updated {script_path}")

Successfully updated /kaggle/working/LLaMA2-Accessory/accessory/exps/finetune/sg/alpaca_llamaPeft_normBiasLora.sh


In [85]:
cat /kaggle/working/LLaMA2-Accessory/accessory/exps/finetune/sg/alpaca_llamaPeft_normBiasLora.sh

#!/bin/bash

pretrained_path=$1
pretrained_type=meta_ori
llama_config="$2 configs/model/finetune/sg/llamaPeft_normBiasLora.json"
tokenizer_path="$3"
data_config=configs/data/finetune/sg/alpaca.yaml

data_parallel=sdp
model_parallel=1

exp_name=finetune/sg/alpaca_llamaPeft_normBiasLora
echo "exp name: $exp_name"
mkdir -p output/"$exp_name"

torchrun --master_port=1112 --nproc_per_node=1 /kaggle/working/LLaMA2-Accessory/accessory/main_finetune.py \
--output_dir output/"$exp_name" --epochs 4 --warmup_epochs 1 \
--batch_size 4 --accum_iter 2 --num_workers 4 \
--max_words 512 \
--lr 0.00005 --min_lr 0.000005 --clip_grad 2 --weight_decay 0.02 \
--data_parallel "$data_parallel" --model_parallel_size "$model_parallel" --checkpointing \
--llama_type llama_peft --llama_config $llama_config --tokenizer_path "$tokenizer_path" \
--no_visual \
--pretrained_path "$pretrained_path" --pretrained_type="$pretrained_type" \
--data_config $data_config \
2>&1 | tee -a output/"$exp_name"/output.log

echo "ex

In [90]:
!exps/finetune/sg/alpaca_llamaPeft_normBiasLora.sh \
  '/kaggle/working/LLaMA2-Accessory/llama-2-7b/consolidated.00.pth' \
  '/kaggle/working/LLaMA2-Accessory/llama-2-7b/params.json' \
  '/kaggle/working/LLaMA2-Accessory/llama-2-7b/tokenizer.model'

exp name: finetune/sg/alpaca_llamaPeft_normBiasLora
2025-04-21 21:21:54.553962: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1745270514.575591     347 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1745270514.581862     347 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
/kaggle/working/LLaMA2-Accessory/accessory/main_finetune.py:41: UserWarning: cannot import FusedAdam from apex, use torch AdamW instead
  warnings.warn("cannot import FusedAdam from apex, use torch AdamW instead")
| distributed init (rank 0): env://, gpu 0
[rank0]:[W421 21:22:00.467947749 ProcessGroupNCCL.cpp:4115] [PG ID 0 PG GUID 0 Rank 0]  using GPU 0 to perform barrier as device

In [79]:
%pwd

'/kaggle/working/LLaMA2-Accessory/accessory'

In [83]:
import torch
print(torch.cuda.device_count())  # Should match what you expect

1
